word2vec tutorial
====

1. 간단한 이론적 소개
2. word2vec 설치
3. 영어 코퍼스로 맛보기
4. gensim으로 요리하기
5. KoNLPy 설치
6. 한국어 코퍼스로 word2vec 생성
7. 의미 코퍼스로 학습 집합 생성
8. 단어 의미 중의성 해결 - Part 1 - 전통적인 방법
9. 단어 의미 중의성 해결 - Part 2 - word2vec 활용

1. 간단한 이론적 소개
----

2. word2vec 설치
----

### 다운로드

```
$ svn checkout http://word2vec.googlecode.com/svn/trunk/ word2vec
```

### 빌드
#### Linux

```
$ cd word2vec
$ make
```

#### Mac
* makefile 내용 수정

```makefile
# CFLAGS = -lm -pthread -O3 -march=native -Wall -funroll-loops -Wno-unused-result
CFLAGS = -lm -pthread -O3 -Wall -funroll-loops    # -march=native, -Wno-unused-result 제거
```

* 소스코드 수정

```
$ grep malloc.h *.c
compute-accuracy.c:#include <malloc.h>
distance.c:#include <malloc.h>
word-analogy.c:#include <malloc.h>
```
malloc.h를 include하는 코드를 모두 지워버립니다. 혹시 아까우시다면 주석 처리를 해도 괜찮아요. ^^;


* 빌드

```
$ make
```

3. 영어 코퍼스로 맛보기
----

* 샘플 코퍼스 다운로드
```
$ wget http://mattmahoney.net/dc/text8.zip
$ unzip text8.zip
```

* word vector 생성

```
$ ./word2vec -train text8 -output text8.w2v -cbow 1 -size 200 -window 8 -negative 25 -hs 0 -sample 1e-4 -threads 4 -binary 1 -iter 15
Starting training using file text8
Vocab size: 71291
Words in train file: 16718843
Alpha: 0.000005  Progress: 100.02%  Words/thread/sec: 92.08k

$ ls -l text8.w2v
-rw-r--r--  1 jamie  staff  57704809 11  6 01:12 text8.w2v
```
-threads argument는 CPU 코어의 갯수에 맞게 적절히 넣으면 됩니다.


* word analogy 해보기

A B C 세 단어를 (소문자로) 넣으면 vector(A) - vector(B) + vector(C)를 계산하여 벡터간 거리로 정렬하여 가장 가까운 단어부터 보여줍니다. (우왕ㅋ굳ㅋ)

```
$ ./word-analogy text8.w2v
Enter three words (EXIT to break): athens greece baghdad

Word: athens  Position in vocabulary: 3068

Word: greece  Position in vocabulary: 1248

Word: baghdad  Position in vocabulary: 7173

                                              Word              Distance
------------------------------------------------------------------------
                                              iran		0.490723
                                             syria		0.488516
                                             egypt		0.458451
                                            turkey		0.450124
                                             basra		0.448245
                                              iraq		0.444451
                                        sassanians		0.438153
                                             mosul		0.431619
                                           lebanon		0.428784
                                         kurdistan		0.426602
                                              sadr		0.423816
                                            persia		0.423356
                                             konya		0.421188
                                           abbasid		0.415041
                                            khitan		0.410358
                                           bukhara		0.410169
                                            kirkuk		0.405520
                                           fatimid		0.404706
                                           iranian		0.404393
                                           ayyubid		0.404103
                                           mameluk		0.403826
                                             libya		0.400661
                                           almohad		0.400090
                                          pakistan		0.398301
                                          iranians		0.397244
                                              arab		0.395184
                                              zmir		0.394183
                                            zagros		0.394126
                                            mashal		0.393248
                                             balkh		0.393002
                                       afghanistan		0.391483
                                           tunisia		0.389862
                                           tahmasp		0.389630
                                        uzbekistan		0.389564
                                            tigris		0.389527
                                        tajikistan		0.389425
                                           emirate		0.388417
                                        kermanshah		0.388197
                                           khanate		0.385721
                                             hejaz		0.384893
```

* 샘플 word analogy 쿼리

```
$ grep -A1 "^:" questions-words.txt
: capital-common-countries
Athens Greece Baghdad Iraq
--
: capital-world
Abuja Nigeria Accra Ghana
--
: currency
Algeria dinar Angola kwanza
--
: city-in-state
Chicago Illinois Houston Texas
--
: family
boy girl brother sister
--
: gram1-adjective-to-adverb
amazing amazingly apparent apparently
--
: gram2-opposite
acceptable unacceptable aware unaware
--
: gram3-comparative
bad worse big bigger
--
: gram4-superlative
bad worst big biggest
--
: gram5-present-participle
code coding dance dancing
--
: gram6-nationality-adjective
Albania Albanian Argentina Argentinean
--
: gram7-past-tense
dancing danced decreasing decreased
--
: gram8-plural
banana bananas bird birds
--
: gram9-plural-verbs
decrease decreases describe describes
```

4. gensim으로 요리하기
----


### gensim 설치

```
$ pip install gensim
```
끝 (우왕ㅋ굳ㅋ)

* 참고
  - https://radimrehurek.com/gensim/models/word2vec.html
  - http://rare-technologies.com/word2vec-tutorial/

In [ ]:
### word2vec 모델을 만들어 봅시다.

import gensim

sentences = sum([line.split() for line in open('text8')], [])    # text8 코퍼스를 모두 읽어들입니다.
print sentences[:10]

# C 프로그램과 유사하게 학습을 실행합니다.
gensim_model = gensim.models.Word2Vec(sentences, size=200, window=8, min_count=5, workers=4, iter=15)

gensim_model.save('text8.w2v.gensim')    # 학습한 모델을 저장합니다.
gensim_new_model = gensim.models.Word2Vec.load('text8.w2v.gensim')    # 저장한 모델을 읽어들입니다.

# 하지만 오래 살기 위해 이러지 말고 Jeff Dean을 믿고 씁시다.

In [ ]:
### 기존에 C 프로그램으로 학습한 모델을 읽어들여 여러가지 연산을 해봅니다.

import gensim

gensim_model = gensim.models.Word2Vec.load_word2vec_format('text8.w2v', binary=True)    # 모델을 읽어들입니다.

# word analogy
print '* woman - man + king =', gensim_model.most_similar(positive=['woman', 'king'], negative=['man'], topn=3)
print

# 미운 오리 새끼를 찾습니다. ;)
print '* The lone duckling =', gensim_model.doesnt_match("breakfast cereal dinner lunch".split())
print

# 두 단어의 유사도를 계산합니다.
print '* Similarity between woman and man =', gensim_model.similarity('woman', 'man')
print

# 단어의 벡터를 출력합니다.
print '* Vector of man:'
print gensim_model['man']

* woman - man + king = [(u'queen', 0.5442705750465393), (u'anjou', 0.48470211029052734), (u'mistress', 0.4786876440048218)]

* The lone duckling = cereal

* Similarity between woman and man = 0.609136053228

* Vector of man:
[  7.33143315e-02   4.47705248e-03  -1.42499268e-01  -2.31839549e-02
   5.60476705e-02   7.34771714e-02  -1.50984228e-02   1.18450247e-01
  -6.05627187e-02   1.03504187e-03  -7.04200566e-03  -2.98108961e-02
   4.88713607e-02   3.99061739e-02   5.11737652e-02   1.13909893e-01
  -5.65241128e-02   1.44870996e-01  -6.46180287e-02   2.84903478e-02
  -1.73272347e-04   2.16623698e-03  -4.42492999e-02   2.15424523e-02
  -9.79404822e-02  -5.70471734e-02  -8.38414803e-02  -3.98078337e-02
  -2.82810424e-02  -5.97659014e-02  -8.99010152e-02  -4.27372977e-02
   1.37358561e-01  -1.16520800e-01  -2.86092572e-02   9.05053783e-03
  -1.10818781e-01   6.26738071e-02  -8.74709710e-03  -6.23363666e-02
   3.06309406e-02   7.23769069e-02   2.64032315e-02  -3.64326164e-02
   2.27875058e-0

5. KoNLPy 설치
----

### konlpy 패키지 설치

```
$ pip install konlpy
```

### mecab-ko 프로그램 설치

* 아래와 같은 마법의 스크립트는 에러가 나니 한땀한땀 해보겠습니다.

```
$ bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
```

* \$HOME/usr/mecab-ko 아래에 설치한다고 가정하고 아래와 같이 소스코드를 다운로드합니다.

```
$ cd $HOME/usr
$ wget https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
```

* 압축을 풀고 빌드/설치를 합니다.

```
$ tar xfz mecab-0.996-ko-0.9.2.tar.gz
$ cd mecab-0.996-ko-0.9.2
$ ./configure --prefix=$HOME/usr/mecab-ko
$ make && make install
```

### mecab-ko 사전 설치

* 역시 \$HOME/usr/mecab-ko 아래에 설치한다고 가정하고 아래와 같이 사전을 다운로드합니다.

```
$ cd $HOME/usr
$ wget https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.0.1-20150920.tar.gz
```

* 압축을 풀고 빌드/설치를 합니다.

```
$ tar xfz mecab-ko-dic-2.0.1-20150920.tar.gz
$ cd mecab-ko-dic-2.0.1-20150920
$ ./autogen.sh
$ ./configure --prefix=$HOME/usr/mecab-ko --with-mecab-config=$HOME/usr/mecab-ko/bin/mecab-config
$ make && make install
```

### 환경 설정

```
$ export LD_LIBRARY_PATH=$HOME/usr/mecab-ko/lib:$LD_LIBRARY_PATH    # Linux의 경우
$ export DYLD_LIBRARY_PATH=$HOME/usr/mecab-ko/lib:$DYLD_LIBRARY_PATH    # Mac의 경우
```

### 테스트

```
$ cd $HOME/usr/mecab-ko
$ bin/mecab
존나좋군?
존나	MAG,*,F,존나,*,*,*,*
좋	VA,*,T,좋,*,*,*,*
군	EF,*,T,군,*,*,*,*
?	SF,*,*,*,*,*,*,*
EOS
```
![](http://pds23.egloos.com/pds/201112/24/41/d0021441_4ef5e7b6eb457.jpg)

### mecab-ko python 모듈 설치

* 소스 코드를 다운로드 합니다.

```
$ cd $HOME/usr
$ git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
```

* 설치 및 빌드를 합니다.

```
$ cd mecab-python-0.996
$ python setup.py build
$ sudo python setup.py install    # virtualenv를 사용하신다면 sudo는 내려놓으셔도 됩니다.
```

In [ ]:
### KoNLPy에서 mecab-ko를 사용해 봅시다.

from konlpy.tag import Mecab
from konlpy.utils import pprint
import os

# 사전 경로를 주지 않으면 /usr/local/lib/mecab/dic/mecab-ko-dic을 찾습니다.
mecab = Mecab('%s/usr/mecab-ko/lib/mecab/dic/mecab-ko-dic' % os.environ['HOME'])
pprint(mecab.pos(u'존나좋군?'))    # [(형태소1, 태그1), (형태소2, 태그2), ...)] 튜플의 리스트 형태로 리턴합니다.

[(존나, MAG),
 (좋, VA),
 (군, EF),
 (?, SF)]


6. 한국어 코퍼스로 word2vec 생성
----

### 원시 코퍼스

* 원시 코퍼스의 압축을 푼다.

```
$ cat raw_corpus.zip.0 raw_corpus.zip.1 >> raw_corpus.zip
$ unzip -q raw_corpus.zip
```

* 인코딩을 UTF-16에서 UTF-8로 변환하여 하나의 파일로 합친다. => raw_corpus.xml

```
$ iconv -f UTF-16 -t UTF-8 raw_corpus/*.txt > raw_corpus.xml
```

In [ ]:
### 태그를 제거하고 문장만 남긴다. => raw_corpus.txt

def strip_tag(line, tag):
    """
    라인을 둘러싼 열고 닫는 xml 태그를 제거하는 함수
    @param  line  xml 태그를 포함하는 라인
    @param  tag   태그명
    @return       태그가 제거된 라인
    """
    open_tag = '<%s>' % tag
    close_tag = '</%s>' % tag
    stripped = line
    if stripped.startswith(open_tag):
        stripped = stripped[len(open_tag):]
    if stripped.endswith(close_tag):
        stripped = stripped[:-len(close_tag)]
    return stripped.strip()

with open('raw_corpus.txt', 'w') as fout:
    for line in open('raw_corpus.xml'):    # 앞서 xml로 저장한 원시 코퍼스를 열어서
        line = line.strip()
        if not line:
            continue
        if line.startswith('<head>') or line.startswith('<p>'):    # <head>, <p> 태그로 시작하는 문장만
            line = strip_tag(line, 'head')
            line = strip_tag(line, 'p')
            if line:
                print >> fout, line    # 한 줄에 한 문장씩 출력

In [ ]:
### KoNLPy를 이용해 품사 태깅을 수행한다. => raw_corpus.tag

from konlpy.tag import Mecab
from konlpy.utils import pprint
import os

mecab = Mecab('%s/usr/mecab-ko/lib/mecab/dic/mecab-ko-dic' % os.environ['HOME'])

with open('raw_corpus.tag', 'w') as fout:
    for line in open('raw_corpus.txt'):    # 앞서 txt로 저장한 원시 코퍼스를 열어서
        for morph, tag in mecab.pos(unicode(line, 'UTF-8')):    # 형태소 분석을 수행한 다음
            print >> fout, '%s/%s' % (morph.encode('UTF-8'), tag.encode('UTF-8')),    # '정치/NNG'의 형태로
        print >> fout    # 한 줄에 한 문장씩 출력

* 앞서 생성한 자동 태깅한 코퍼스를 이용하여 '단어/태그'의 벡터를 생성한다. => raw_corpus.w2v

```
$ ./word2vec -train raw_corpus.tag -output raw_corpus.w2v -cbow 1 -size 200 -window 8 -negative 25 -hs 0 -sample 1e-4 -threads 4 -binary 1 -iter 15
```

7. 의미 코퍼스로 학습 집합 생성
----

### 의미 코퍼스

* 의미 코퍼스의 압축을 푼다.

```
$ cat sense_corpus.zip.0 sense_corpus.zip.1 >> sense_corpus.zip
$ unzip -q sense_corpus.zip
```

* 인코딩을 UTF-16에서 UTF-8로 변환하여 하나의 파일로 합친다. => sense_corpus.xml

```
$ iconv -f UTF-16 -t UTF-8 sense_corpus/*.txt > sense_corpus.xml
```

In [ ]:
### 특정 단어를 포함하는 문장을 추출하여 단어의 의미 번호와 함께 출력한다. => sense_corpus.tag

TARGET = '연기__'    # 추출을 원하는 단어. 예: 배, 경기, 연기

import re

LINE_ID_PTN = re.compile('[0-9A-Z_]{4}\\d{4}-\\d{7,8}')    # 형태소 분석 된 어절 한 줄의 맨 앞에 ID 패턴

with open('sense_corpus.tag', 'w') as fout:
    sent = []
    for line in open('sense_corpus.xml'):    # 앞서 xml로 저장한 의미 코퍼스를 열어서
        line = line.strip()
        if not line:
            continue
        if LINE_ID_PTN.match(line):    # 다른 라인은 버리고 어절만을 취하여
            _, morphs = line.rsplit('\t', 1)
            sent.extend(morphs.split(' + '))
        else:
            # 한 문장에서 원하는 단어의 의미 번호를 모두 추출하여
            sense_nums = set([morph[len(TARGET):morph.rindex('/')] for morph in sent if morph.startswith(TARGET)])
            if len(sense_nums) == 1:    # 한 문장에 원하는 단어의 의미가 한가지로만 쓰인 문장만을 골라서
                try:
                    sense_num = int(list(sense_nums)[0])
                    if sense_num < 50:    # 의미 번호가 너무 크면 뭔가 에러가 있으므로 버리고
                        # 단어에 붙어있는 모든 의미 번호를 제거하고
                        no_sense_tag = [re.sub(r"""__x?\d\d/""", '/', morph) for morph in sent]
                        # 문장 맨 앞에 원하는 단어의 의미 번호를 함께 출력
                        print >> fout, '%d\t%s' % (sense_num, ' '.join(no_sense_tag))
                except ValueError:
                    # 간혹 의미 번호가 '연기__x01'과 같이 특수하게 부여된 것이 있는데, 이런건 버린다.
                    pass
            sent = []

8. 단어 의미 중의성 해결 - Part 1 - 전통적인 방법
----

In [ ]:
### 의미 코퍼스를 읽어들입니다.

sense_corpus = []
for line in open('sense_corpus.tag'):
    sense, features = line.split('\t', 1)
    sense_corpus.append((int(sense), features.split()))

In [ ]:
### 전부 바이너리 벡터로 표현합니다.

import numpy as npy

def to_bin_vector(vocabulary, features, trg):
    """
    vocabulary 사전을 이용하여 모든 자질을 하나의 bag-of-words 벡터로 표현하는 함수
    @param  vocabulary  vocabulary 사전
    @param  features    자질 목록
    @param  trg         중의성을 해소할 단어
    @return             벡터
    """
    vec_sum = npy.zeros([len(vocabulary),], dtype=npy.int8)    # vocabulary 크기 만큼 벡터를 0으로 초기화 합니다.
    for feature in features:
        vec_sum[vocabulary[feature]] = 1    # 자질 번호에 해당하는 벡터 값을 1로 설정해 줍니다.
    return vec_sum

all_features = set(sum([features for _, features in sense_corpus], []))    # 모든 자질을 합칩니다.
vocabulary = {feature: num for num, feature in enumerate(all_features)}    # 각 자질에 고유 번호를 부여하여 사전을 구축합니다.
print 'Number of vocabulary:', len(vocabulary)

# 자질을 전부 벡터로 표현합니다.
bin_train = [(sense, to_bin_vector(vocabulary, features, '연기')) for sense, features in sense_corpus]

Number of vocabulary: 7628


In [ ]:
### SVM을 이용하여 학습 및 평가를 수행하는 함수를 작성합니다.

from sklearn import cross_validation
from sklearn import svm

def evaluate(X, Y):
    """
    SVM을 이용하여 평가를 수행하는 함수
    @param  X  자질 벡터
    @param  Y  outcome
    @return    정확도
    """
    # 학습 코퍼스를 8:2의 비율로 학습/평가 코퍼스로 나눕니다.
    X_train, X_eval, y_train, y_eval = cross_validation.train_test_split(X, Y, test_size=0.2, random_state=0)
    svm_model = svm.SVC()
    svm_model.fit(X_train, y_train)    # 학습 집합을 이용해 학습합니다.
    return svm_model.score(X_eval, y_eval)    # 평가 집합으로 정확도를 평가합니다.

In [ ]:
### 평가를 해봅니다.

evaluate([features for _, features in bin_train], [sense for sense, _ in bin_train])

# oTL ㅠ.ㅠ

0.39393939393939392

![](https://pbs.twimg.com/profile_images/2104851627/image.jpg)

9. 단어 의미 중의성 해결 - Part 2 - word2vec 활용
----

In [ ]:
### 전부 word2vec을 이용하여 벡터로 표현합니다.

import gensim
import numpy as npy

def to_w2v_vector(model, features, trg):
    """
    word2vec 모델을 이용하여 모든 자질을 더해서 하나의 벡터로 표현하는 함수
    @param  model     word2vec 모델
    @param  features  자질 목록
    @param  trg       중의성을 해소할 단어
    @return           벡터
    """
    vec_sum = npy.zeros([model.vector_size,])    # word2vec 모델의 벡터 크기 만큼 벡터를 0으로 초기화 합니다.
    for feature in set(features):
        try:
            vec_sum += model[feature]
        except KeyError:
            pass    # Out-Of-Vocabulary 자질들은 그냥 넘어갑니다.
    return vec_sum

# 원시 코퍼스로 pre-training 해둔 word2vec 모델을 읽어들입니다.
w2v_model = gensim.models.Word2Vec.load_word2vec_format('raw_corpus.w2v', binary=True)
print 'Vector size:', w2v_model.vector_size

# 자질을 전부 벡터로 표현합니다.
w2v_train = [(sense, to_w2v_vector(w2v_model, features, '연기')) for sense, features in sense_corpus]

Vector size: 200


In [ ]:
### 평가를 해봅니다.

evaluate([features for _, features in w2v_train], [sense for sense, _ in w2v_train])

# Part 1보다는 좀 낫지만 여전히 oTL ㅠ.ㅠ

0.43097643097643096

![](https://cdn.mirror.wiki/http://cdn.mirror.wiki/https://attachment.namu.wiki/otl.jpg)